In [2]:
from pymongo import MongoClient
import sshtunnel
from bson import ObjectId
from rich import print_json
from rich.pretty import pprint
from rich.progress import track, Progress, TextColumn, BarColumn, TimeElapsedColumn
from rich.table import Column
import os.path

def print_mongo(obj):
    obj=obj.copy()
    obj['_id']=str(obj['_id'])
    print_json(data=obj)

import pydantic
from mupifDB import models
coll2model={
    'WorkflowExecutions':models.WorkflowExecution_Model,
    'Workflows':models.Workflow_Model,
    'UseCases':models.UseCase_Model,
    'IOData':models.IODataRecord_Model,
    'WorkflowsHistory':models.Workflow_Model,
}
vpns={'test':'172.24.1.1','musicode':'172.22.2.1','deema':'172.23.1.1','test6':'fd4e:6fb7:b3af::1','sumo':'172.25.1.1','tinnit':'172.26.1.1'}

with Progress(TextColumn('[progress.description]{task.description}',table_column=Column(width=20)),TextColumn('{task.completed}/{task.total}',table_column=Column(width=15)),BarColumn(),TimeElapsedColumn()) as progress:
    vpn_task=progress.add_task('[orchid1]…',total=len(vpns))
    col_task=progress.add_task(' [green]…',start=False)
    doc_task=progress.add_task('  [cyan]…',start=False)
    for vpn,node in vpns.items():
        progress.update(vpn_task,description=f'[orchid1]{vpn}')
        progress.update(col_task,description='↳[green]…',start=False)
        progress.update(doc_task,description=' ↳[cyan]…',start=False)
        with sshtunnel.open_tunnel(
            (node,22),
            ssh_username='root',
            ssh_pkey=os.path.expanduser('~/.ssh/id_rsa'),
            remote_bind_address=('localhost',27017),
            local_bind_address=('localhost',0),
            ssh_config_file=None,
        ) as tunnel:
            client = MongoClient(f'mongodb://localhost:{tunnel.local_bind_port}')
            # local_bind_address=('0.0.0.0', 10022)
            db = client.MuPIF
            progress.reset(col_task,start=True)
            progress.update(col_task,description='↳[green]…',total=len(coll2model),completed=0)
            progress.update(doc_task,description=' ↳[cyan]…',complete=None,total=None)
            for coll,Model in coll2model.items():
                progress.update(col_task,description=f'↳[green]{coll}…')
                progress.reset(doc_task,description=' ↳[cyan](querying)…',start=False,refresh=True)
                c=db.get_collection(coll)
                cursor=c.find()
                progress.update(doc_task,total=c.count_documents(filter={}),description=f' ↳[cyan]{coll}')
                progress.start_task(doc_task)
                for i,obj in enumerate(cursor):
                    try: Model.model_validate(obj)
                    except pydantic.ValidationError:
                        print_mongo(obj)
                        raise
                    progress.advance(doc_task)
                    if vpn=='sumo' and i>10000: break
                progress.advance(col_task)
        progress.advance(vpn_task)
        progress.refresh()
                # print(f'{coll}: {i} done')


Output()